In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd.profiler as profiler

import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms


LOG_PATH = "./data/profile_info/"
DATA_DIR = './data/MNIST'

In [ ]:
SCALED_SHAPE = (160, 160)

In [3]:
def load_data(data_dir, batch_size, transform):
    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size)
    test_loader = torch.utils.data.DataLoader(testset, batch_size)

    return train_loader, test_loader

In [ ]:
default_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

scaled_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(SCALED_SHAPE),
    transforms.Normalize((0.5,), (0.5,)),
])

default_train_loader, default_test_loader = load_data(data_dir=DATA_DIR, batch_size=batch_size, transform=default_transform)
scaled_train_loader, scaled_test_loader = load_data(data_dir=DATA_DIR, batch_size=batch_size, transform=scaled_transform)

In [4]:
in_channels = 3
device = "cuda:0"

#train_loss = nn.CrossEntropyLoss()

learning_rate = 1e-4

batch_size = 256
epoch_num = 25

In [6]:
def train(net, optimizer, criterion, epoch_num, default_train_data, scaled_train_data, device):
    """Neural network training process.
    
    """
    for epoch in range(epoch_num):  # loop over the dataset multiple times
        for ((default_data, _), (scaled_data, _)) in zip(default_train_data, scaled_train_data):
            inputs = default_data.to(device)
            masks = scaled_data.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()    
    return net

In [7]:
class Network(nn.Module):
    def __init__(self, channels_list):
        super().__init__()
        channels_list.insert(0, 1)
        self.downsample = nn.ModuleList([
            nn.Sequential(
                nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU()
            )
            for in_ch, out_ch in zip(channels_list[:-1], channels_list[1:])
        ])
        self.upsample = nn.ModuleList([
            nn.Sequential(
                nn.ConvTranspose2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_ch),
                nn.ReLU()
            )
            for in_ch, out_ch in zip(channels_list[:-1:-1], channels_list[1::-1])
        ])


    def forward(self, tensor):
        for layer in self.downsample:
            tensor = layer(tensor)
            tensor = F.max_pool2d(tensor, kernel_size=2) 
        for layer in self.upsample:
            tensor = layer(tensor)       
        
        return tensor

In [8]:
net = Network([8, 16, 32])
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [9]:
%%time
net = train(
    net, optimizer, train_loss, epoch_num, trainset, device
)

CPU times: user 4d 10h 25min 43s, sys: 28min 58s, total: 4d 10h 54min 42s
Wall time: 5h 24min 24s


In [10]:
%%time
for data, _ in testset:
    inputs = data.to(device)
    _ = net(inputs)

CPU times: user 20min 15s, sys: 4.14 s, total: 20min 19s
Wall time: 45.4 s
